# Create a sphere using PyVista

This notebook creates a sphere using PyVista. There are 3 different iterations:

1. Use PyVista's Delaunay triangulation method to create a sphere.
2. Construct a sphere from scratch (ish) by generating arrays for vertices and vertex normals, and using `scipy.spatial.ConvexHull` to generate a triangles array. For this iteration, the vertices are not uniformly distributed over the sphere (the vertices are generated using spherical coordinates).
3. Construct a sphere from scratch (ish) by generating arrays for vertices and vertex normals, and using `scipy.spatial.ConvexHull` to generate a triangles array. For this iteration, the vertices are uniformly distributed over the sphere (the vertices are generated using Cartesian coordinates).


In [1]:
import numpy as np
import pyvista as pv
from scipy.spatial import ConvexHull

## 1 - Generate a cube using PyVista Delaunay triangulation.


### 1.1 - Vertices

We want to generate an array of vertices with the correct shape.


Make arrays of $r$, $\theta$, $\phi$ values.


In [2]:
r = np.array([1])
theta = np.linspace(0, np.pi, 10)
phi = np.linspace(0, 2 * np.pi, 20)

Convert from spherical coordinates to Cartesian coordinates.


In [3]:
r_v, theta_v, phi_v = np.meshgrid(np.array([1]), theta, phi)

x_v = r_v * np.sin(theta_v) * np.cos(phi_v)
y_v = r_v * np.sin(theta_v) * np.sin(phi_v)
z_v = r_v * np.cos(theta_v)

Combine the $x$, $y$, $z$ values into an array, `vertices`, with the correct shape. `vertices` should have shape ($NM$, 3), where $N$, $M$ is the number of $\theta$, $\phi$ values.


In [4]:
vertices = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

vertices.shape

(200, 3)

### 1.2 - PyVista mesh

We want to create a PyVista mesh.


Create and plot the PyVista mesh. This mesh only has vertices (no faces).


In [5]:
mesh = pv.PolyData(vertices)
mesh.plot()

Widget(value='<iframe src="http://localhost:56134/index.html?ui=P_0x13596f4d0_0&reconnect=auto" class="pyvista…

### 1.3 - Connected surface

We want to create a connected surface.


Create and plot a connected surface, using Delaunay triangulation.


In [6]:
surface = mesh.delaunay_3d()
surface.plot(show_edges=True)

Widget(value='<iframe src="http://localhost:56134/index.html?ui=P_0x144d51450_1&reconnect=auto" class="pyvista…

## 2 - Generate a sphere from scratch (ish).


### 2.1 - Vertices

We want to generate an array of vertices with the correct shape.


Make arrays of $r$, $\theta$, $\phi$ values. $N$ is the number of polar angles sampled.


In [302]:
N = 100

r = np.array([1])

cos_theta = np.linspace(-1, 1, N)
theta = np.arccos(cos_theta)

# sin_theta = np.linspace(-1, 1, N)
# theta = np.arccos(sin_theta)

# theta = np.linspace(0, np.pi, N)

phi = np.linspace(0, 2 * np.pi, 2 * N)

Convert from spherical coordinates to Cartesian coordinates.


In [303]:
r_v, theta_v, phi_v = np.meshgrid(np.array([1]), theta, phi)

x_v = r_v * np.sin(theta_v) * np.cos(phi_v)
y_v = r_v * np.sin(theta_v) * np.sin(phi_v)
z_v = r_v * np.cos(theta_v)

Combine the $x$, $y$, $z$ values into an array, `vertices`, with the correct shape. `vertices` should have shape ($NM$, 3), where $N$, $M$ is the number of $\theta$, $\phi$ values. Remove duplicate values from `vertices`.


In [304]:
vertices = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]
vertices = np.unique(vertices, axis=0)

vertices.shape

(19801, 3)

### 2.2 - Vertex normals

We want to generate an array of vertex normals with the correct shape.


Calculate the normals.


In [305]:
center = np.array([0, 0, 0])
normals = vertices - center[None, :]

Normalize the normals. Similarly to `vertices`, `normals` should have shape ($NM$,3).


In [306]:
normals = normals / np.linalg.norm(normals, axis=1)[:, None]

normals.shape

(19801, 3)

### 2.3 - Triangles

We want to generate an array of triangles.


The triangulation is handled by SciPy's `ConvexHull` method. The format of the triangle array, `triangles`, is `[[v1, v2, v3], [v4, v5, v6], ...]`.


In [307]:
triangles = ConvexHull(vertices).simplices

triangles.shape

(39004, 3)

### 2.4 - PyVista mesh

We want to create the PyVista mesh.


PyVista expects the triangles to be given in the format `[3, v1, v2, v3, 3, v4, v5, v6, ...]`. Reformat `triangles`.


In [308]:
triangles_reformatted = np.zeros((triangles.shape[0], 4), dtype=int)
triangles_reformatted[:, 0] = 3
triangles_reformatted[:, 1:] = triangles

Flatten `triangles_reformatted`.


In [309]:
triangles_flattened = triangles_reformatted.ravel()

triangles_flattened.shape

(156016,)

Create the PyVista mesh, and add the normals as point data. The mesh should have $NM$ vertices.


In [310]:
cube_mesh = pv.PolyData(vertices, triangles_flattened)
cube_mesh.point_data["normals"] = normals

print(f"Number of vertices: {cube_mesh.n_points}")
print(f"Number of cells: {cube_mesh.n_cells}")

Number of vertices: 19801
Number of cells: 39004


### 2.5 - Plot the mesh

We want to plot the PyVista mesh.


Plot the PyVista mesh, along with vertex normals.


In [311]:
pl = pv.Plotter()

pl.add_mesh(cube_mesh, show_edges=True, color="lightblue")
arrows = cube_mesh.glyph(orient="normals", scale=False, factor=0.1)
pl.add_mesh(arrows, color="red")

pl.show()

Widget(value='<iframe src="http://localhost:57828/index.html?ui=P_0x39ffa9590_24&reconnect=auto" class="pyvist…

## 3 - Generate a sphere from scratch (ish), and distribute the vertices uniformly.


### 3.1 - Vertices

We want to generate an array of vertices with the correct shape.


Make arrays of $x$, $y$ values. $N$ is the number of vertices per side.


In [149]:
N = 300

x = np.linspace(-1, 1, N)
y = np.linspace(-1, 1, N)

x_v, y_v = np.meshgrid(x, y)
xy_coordinates = np.c_[x_v.reshape(-1), y_v.reshape(-1)]

xy_coordinates.shape

(90000, 2)

Remove coordinate pairs which don't satisfy $x^2 + y^2 \leq 1$.


In [150]:
radius_squared = xy_coordinates[:, 0] ** 2 + xy_coordinates[:, 1] ** 2
mask = radius_squared <= 1.0
xy_coordinates = xy_coordinates[mask]
radius_squared = radius_squared[mask]

xy_coordinates.shape

(70168, 2)

Create an array, `vertices`, to store the vertices. Populate it with the $(x, y)$ coordinates (twice).


In [151]:
num_coords = xy_coordinates.shape[0]
vertices = np.zeros((num_coords * 2, 3), dtype=float)

vertices[0:num_coords, 0:2] = xy_coordinates
vertices[num_coords : 2 * num_coords, 0:2] = xy_coordinates

vertices.shape

(140336, 3)

Calculate the $z$ coordinates for each $(x, y)$ coordinate using the formula $z = \pm \sqrt{1 - x^2 - y^2}$. Remove duplicate vertices.


In [ ]:
vertices[0:num_coords, 2] = np.sqrt(1 - radius_squared)
vertices[num_coords : 2 * num_coords, 2] = vertices[0:num_coords, 2] * -1
vertices = np.unique(vertices, axis=0)

vertices.shape

(140336, 3)

### 3.2 - Vertex normals

We want to generate an array of vertex normals with the correct shape.


Calculate the normals.


In [153]:
center = np.array([0, 0, 0])
normals = vertices - center[None, :]

Normalize the normals. Similarly to `vertices`, `normals` should have shape ($NM$,3).


In [154]:
normals = normals / np.linalg.norm(normals, axis=1)[:, None]

normals.shape

(140336, 3)

### 3.3 - Triangles

We want to generate an array of triangles.


The triangulation is handled by SciPy's `ConvexHull` method. The format of the triangle array, `triangles`, is `[[v1, v2, v3], [v4, v5, v6], ...]`.


In [155]:
triangles = ConvexHull(vertices).simplices

triangles.shape

(280668, 3)

### 3.4 - PyVista mesh

We want to create the PyVista mesh.


PyVista expects the triangles to be given in the format `[3, v1, v2, v3, 3, v4, v5, v6, ...]`. Reformat `triangles`.


In [156]:
triangles_reformatted = np.zeros((triangles.shape[0], 4), dtype=int)
triangles_reformatted[:, 0] = 3
triangles_reformatted[:, 1:] = triangles

Flatten `triangles_reformatted`.


In [157]:
triangles_flattened = triangles_reformatted.ravel()

triangles_flattened.shape

(1122672,)

Create the PyVista mesh, and add the normals as point data. The mesh should have $NM$ vertices.


In [158]:
cube_mesh = pv.PolyData(vertices, triangles_flattened)
cube_mesh.point_data["normals"] = normals

print(f"Number of vertices: {cube_mesh.n_points}")
print(f"Number of cells: {cube_mesh.n_cells}")

Number of vertices: 140336
Number of cells: 280668


### 3.5 - Plot the mesh

We want to plot the PyVista mesh.


Plot the PyVista mesh, along with vertex normals.


In [159]:
pl = pv.Plotter()

pl.add_mesh(cube_mesh, show_edges=True, color="lightblue")
arrows = cube_mesh.glyph(orient="normals", scale=False, factor=0.15)
pl.add_mesh(arrows, color="red")

pl.show()

Widget(value='<iframe src="http://localhost:57828/index.html?ui=P_0x30c19c2d0_8&reconnect=auto" class="pyvista…